In [40]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API")

llm = ChatGroq(
    api_key=groq_api_key,
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

In [42]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages

In [47]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

def chat_node(state: ChatState):
    # now state should have 'messages'
    messages = state.get('messages', [])  # safer
    response = llm.invoke(messages)
    # accumulate history
    print(messages)
    return {'messages': messages + [response]}


In [48]:
graph = StateGraph(ChatState)
graph.add_node('chat_node', chat_node)
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

In [49]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of India')]
}

result = chatbot.invoke(initial_state)
print(result['messages'][-1].content)


[HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='40aa066a-cec5-42d9-970b-1a3e4d250cd6')]
The capital of India is New Delhi. It is a part of the larger city of Delhi and serves as the administrative and governmental center, housing the central government institutions. New Delhi was established by the British as the capital and has remained so since India's independence.


In [50]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting chat.")
        break
    initial_state = {
        'messages': result['messages'] + [HumanMessage(content=user_input)]
    }
    result = chatbot.invoke(initial_state)
    print("Bot:", result['messages'][-1].content)

[HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='40aa066a-cec5-42d9-970b-1a3e4d250cd6'), AIMessage(content="The capital of India is New Delhi. It is a part of the larger city of Delhi and serves as the administrative and governmental center, housing the central government institutions. New Delhi was established by the British as the capital and has remained so since India's independence.", additional_kwargs={'reasoning_content': 'Okay, so I need to figure out what the capital of India is. Hmm, I\'m not entirely sure, but I think it\'s a city that starts with an \'N\'. Maybe New Delhi? I\'ve heard that name before in the context of India. But wait, is New Delhi the capital, or is it just a part of the capital? I remember learning that sometimes countries have a federal district as their capital, like Washington D.C. in the United States. So perhaps New Delhi is part of a larger area that serves as the capital.\n\nLet me think about ot

## ---------------------------------------   check pointe instead of stacking manually 

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API")

llm = ChatGroq(
    api_key=groq_api_key,
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

def chat_node(state: ChatState):
    # assume state['messages'] already has history if any
    messages = state.get('messages', [])
    response: AIMessage = llm.invoke(messages)
    # return only new message; use add_messages to append
    return {'messages': [response]}

# build graph
builder = StateGraph(ChatState)
builder.add_node('chat_node', chat_node)
# add edges
builder.add_edge(START, 'chat_node')
builder.add_edge('chat_node', END)

# use MemorySaver checkpointer
memory = MemorySaver()
chatbot = builder.compile(checkpointer=memory)

# when invoking
initial_state = {
    'messages': [HumanMessage(content='What is the capital of India')]
}

config = {"configurable": {"thread_id": "thread1"}}
output = chatbot.invoke(initial_state, config=config)
print(output['messages'][-1].content)

# second invocation using same thread_id will load the past messages automatically


The capital of India is New Delhi. It is a part of the larger city of Delhi and serves as the administrative and governmental center, housing the central government institutions. New Delhi was established by the British as the capital and has remained so since India's independence.


In [2]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='What is the capital of India', additional_kwargs={}, response_metadata={}, id='4d9ec78f-9ce6-408c-b9b0-e25edcad109f'), AIMessage(content="The capital of India is New Delhi. It is a part of the larger city of Delhi and serves as the administrative and governmental center, housing the central government institutions. New Delhi was established by the British as the capital and has remained so since India's independence.", additional_kwargs={'reasoning_content': 'Okay, so I need to figure out what the capital of India is. Hmm, I\'m not entirely sure, but I think it\'s a city that starts with an \'N\'. Maybe New Delhi? I\'ve heard that name before in the context of India. But wait, is New Delhi the capital, or is it just a part of the capital? I remember learning that sometimes countries have a federal district as their capital, like Washington D.C. in the United States. So perhaps New Delhi is part of a larger area that serves as the